# EpiRC-PGSの実装

[Near-Optimal Policy Identification in Robust Constrained Markov Decision Processes via Epigraph Form](https://openreview.net/forum?id=G5sPv4KSjR)において提案された，**Epigraph Robust Constrained Policy Gradient Search**(EpiRC-PGS)の再現実装を行いました．

* ほとんど自力で作ったので，ひどいコードです．
* 外側ループの二分探索は正しいと思います．
* 内側ループの勾配収束が現状の問題です．
  * 内側ループ内の反復で方策がすぐに決定論的になってしまって，勾配が収束しない問題が発生しているっぽいです．直したいです．
* numpyで実装したので反復計算が結構遅いです．jaxで描いたほうがいいかも
* コメントは適宜改良します．




In [ ]:
import random
from jax import grad
import numpy as np

S = 3 # 状態数
A = 2 # 行動数
N = 4  # 制約+目的関数の数
T = 100  # エピソード数
U_num = 3 # 不確実性集合の要素数
alpha = 0.01  # 学習率
gamma = 0.95 # 割引率

# 方策の初期化
# pi は S*A の1次元ベクトルとして初期化(論文上でベクトルだけど，コードなら行列にしたほうが良かった気がする)
pi = np.zeros((S, A)) 
pi_temp_table = np.random.rand(S, A)
pi_temp_table = pi_temp_table / np.sum(pi_temp_table, axis=1, keepdims=True)
pi = pi_temp_table.flatten() # グローバル変数 pi (1次元 S*A ベクトル)

# 初期状態分布 mu の初期化
# mu は S 次元のベクトルとして初期化
mu = np.random.rand(S)
mu = mu / np.sum(mu) 

# 有効ホライゾンの定義
H = int(np.round(1 / (1 - gamma)))

# 不確実性集合の作成
def make_U():
    U_list = []
    for _ in range(U_num):
        P_kernel = np.random.rand(S, A, S)
        P_kernel = P_kernel / np.sum(P_kernel, axis=2, keepdims=True)
        U_list.append(P_kernel)
    return U_list

U = make_U() 

# N個(制約数個)のコスト関数、初期化されたQ行列、閾値を作成
def make_C_and_Q_pi_b():
    C_list = []
    Q_list = [] 
    B_list = []
    for _ in range(N): 
        c_sa = np.random.rand(S, A)
        C_list.append(c_sa)
        Q_pi_initial = np.zeros((S, A)) 
        Q_list.append(Q_pi_initial)
        b_val = random.randint(0, H)
        B_list.append(b_val)
    return C_list, Q_list, B_list

C, Q_initial_list, B = make_C_and_Q_pi_b() 
                                       


def compute_dot_pi_Q(s_idx, current_Q_sa,pi): 
    """
    方策 pi と Q 関数の内積を計算
    pi は S*A の1次元ベクトルとして与えられ、Qは S*A の2次元行列として与えられる。
    input:
    s_idx: 状態のインデックス (0 <= s_idx < S)
    current_Q_sa: Q関数の行列 (S, A)
    pi: 方策の1次元ベクトル (S*A)
    output:
    V_s: 状態 s の価値関数 V(s) の値
    """

    pi_s_actions = pi[s_idx * A : (s_idx + 1) * A] #s_idx行における行動列ベクトルの獲得．次元は A
    return np.dot(pi_s_actions, current_Q_sa[s_idx, :])# Q行列のs_idx行と行動列ベクトルの内積を計算．スカラー値

def compute_sum_dot_pi_Q(current_Q_sa,pi): 
    """
    全てのsについて方策とQ関数の内積を計算した配列
    
    input:
    current_Q_sa: Q関数の行列 (S, A)
    pi: 方策の1次元ベクトル (S*A)
    output:
    V_s_results: 各状態 s の価値関数 V(s) の値を格納した配列 (S,)
    """

    V_s_results = np.zeros(S) # 出力用S次元のベクトルを初期化
    for s_idx in range(S): 
        V_s_results[s_idx] = compute_dot_pi_Q(s_idx, current_Q_sa,pi) # Q行列のs_idx行と方策ののs_idx行動列ベクトルの内積を計算し，s_idx行に格納
    return V_s_results # 各状態 s の価値関数V(s)の値を格納した配列．次元はS

def compute_sum_P_dot_dot_pi_Q(s_idx, a_idx, P_kernel_sas, current_Q_sa,pi):
    """
    sum_{s'} P(s'|s,a) V(s') を計算
    input:
        s_idx: 状態のインデックス (0 <= s_idx < S)
        a_idx: 行動のインデックス (0 <= a_idx < A)
        P_kernel_sas: 状態遷移確率行列 (S, A, S')
        current_Q_sa: Q関数の行列 (S, A)
        pi: 方策の1次元ベクトル (S*A)
    output:
        sum_{s'} P(s'|s,a) V(s')，スカラー
    """
    V_s_prime_current = compute_sum_dot_pi_Q(current_Q_sa,pi) # Vを計算
    return np.dot(P_kernel_sas[s_idx, a_idx, :], V_s_prime_current)#VとPの内積をとる．

def compute_Q_pi_c_P(s_idx, a_idx, P_kernel_sas, current_Q_sa, cost_sa_current,pi):
    """
    Q(s,a) の1ステップ更新値を計算します(ベルマン方程式の適用)

    Q^\pi_c_P(s,a) = c(s,a) + gamma * sum_{s'} P(s'|s,a) V(s')

    input:
        s_idx: 状態のインデックス (0 <= s_idx < S)
        a_idx: 行動のインデックス (0 <= a_idx < A)
        P_kernel_sas: 状態遷移確率行列 (S, A, S')
        current_Q_sa: Q関数の行列 (S, A)
        cost_sa_current: コスト関数 (S, A)
        pi: 方策の1次元ベクトル (S*A)
    output:
        Q(s,a) の1ステップ更新値，スカラー
    """
    expected_future_val = compute_sum_P_dot_dot_pi_Q(s_idx, a_idx, P_kernel_sas, current_Q_sa,pi)
    return cost_sa_current[s_idx, a_idx] + gamma * expected_future_val

def compute_Q_pi_c_P_matrix(P_kernel_sas, Q_sa_initial_guess, cost_sa_current,pi, max_iterations=1000,tolerance=0.001):
    """
    Q関数の行列を反復計算します．ベルマン方程式を更新誤差が収束するまで反復的に計算します．
    input:
        P_kernel_sas: 状態遷移確率行列 (S, A, S')
        Q_sa_initial_guess: Q関数の初期値 (S, A)
        cost_sa_current: コスト関数 (S, A)
        pi: 方策の1次元ベクトル (S*A)
        max_iterations: 最大反復回数
        tolerance: 収束判定の閾値
    output:
        Q_k_plus_1_sa: 収束したQ関数の行列 (S, A)
    """
    Q_k_sa = np.copy(Q_sa_initial_guess)

    
    # ベルマン方程式で更新されたQを反復的にベルマン方程式を適用して計算
    for iteration in range(max_iterations):
        Q_k_plus_1_sa = np.zeros((S, A)) 
        # Qの行列全てに対してベルマン方程式を適用
        for s_idx in range(S):
            for a_idx in range(A):
                Q_k_plus_1_sa[s_idx, a_idx] = compute_Q_pi_c_P( # 元の compute_Q_pi_c_P を呼び出し
                    s_idx, a_idx, P_kernel_sas, Q_k_sa, cost_sa_current,pi
                )
        
        # 収束判定
        if np.max(np.abs(Q_k_plus_1_sa - Q_k_sa)) < tolerance:
            print(f"Q converged at iteration {iteration + 1} for P: {P_kernel_sas[0,0,0]:.2f} c: {cost_sa_current[0,0]:.2f}")
            return Q_k_plus_1_sa # 収束したQ行列を返す
        Q_k_sa = Q_k_plus_1_sa 
    return Q_k_sa # 収束しなかった場合は最後のQ行列を返す

def compute_J_c_P(P_kernel_sas, Q_sa_initial_guess, cost_sa_current,pi):
    """
    目的関数 J_c_P を計算します。
    
    Parameters:
    P_kernel_sas : np.ndarray
        状態遷移確率行列
    Q_sa_initial_guess : np.ndarray
        Q行列の初期値
    cost_sa_current : np.ndarray
        コスト関数

    Returns:
    float
        目的関数の値
    """
    Q_sa = compute_Q_pi_c_P_matrix(P_kernel_sas, Q_sa_initial_guess, cost_sa_current,pi)  # Q行列を計算
    V_s = np.zeros(S)  
    pi_matrix = pi.reshape(S, A)  
    for s_idx in range(S):
        V_s[s_idx] = np.dot(pi_matrix[s_idx, :], Q_sa[s_idx, :]) 
    return np.dot(mu, V_s) 

def compute_Q_pi_for_all_U(C, Q_initial_list,U,pi):  
    Q_cn_U_array = np.zeros((N, U_num),dtype=object) 
    for j in range(N):
        Q_results_respect_U = []  
        for i in range(U_num):
            Q_result = compute_Q_pi_c_P_matrix(U[i], Q_initial_list[j], C[j],pi)
            Q_results_respect_U.append(Q_result)
            Q_cn_U_array[j, i] = Q_result 

    return Q_cn_U_array  

def compute_J_c_U_b_and_its_max(U, C, Q_initial_list, B,pi):
    """
    各不確実性集合に対して目的関数を計算し、最大値を返します。
    
    Parameters:
    U : list of np.ndarray
        不確実性集合のリスト
    C : list of np.ndarray
        コスト関数のリスト
    Q_initial_list : list of np.ndarray
        Q行列の初期値のリスト
    B : list of float
        制約条件のリスト

    Returns:
    tuple
        目的関数の値とその最大値
    """
    J_results = []
    J_max_index_U = [] 
    for n in range(N)
        J_results_respect_U = []
        for i in range(U_num):
            J_result = compute_J_c_P(U[i], Q_initial_list[n], C[n], pi)   
            J_results_respect_U.append(J_result)
        J_max_index_U.append(np.argmax(J_results_respect_U))  
        J_results.append(np.max(J_results_respect_U)- B[n])  
    # argmaxをとる
    J_max_index = np.argmax(J_results)
    return J_results, J_max_index,J_max_index_U  


def compute_occupancy_measure_d_P_pi(
    P_kernel_sas,             
    current_gamma,
    initial_dist_mu_s,     
    num_states,
    num_actions,
    pi
    ):
    """ d_P^pi(s) を計算 """
    
    pi_table_sa = pi.reshape(num_states, num_actions)  # S行A列の形に変形
    
    # 1. 実効的な状態遷移確率 P_pi(s'|s) = sum_a pi(a|s)P(s'|s,a) を計算
    # P_kernel_sas: (S, A, S') -> ijk
    # pi_table_sa: (S, A) -> ij
    # 結果 P_pi_ss_prime: (S, S') -> ik
    P_pi_ss_prime = np.einsum('ijk,ij->ik', P_kernel_sas, pi_table_sa)
    # 2. 線形方程式 (I - gamma * (P_pi)^T) d^T = (1-gamma) mu^T を解く
    #    d は行ベクトルとして扱いたいので、転置して d^T を求める
    #    (A_matrix) x = b  =>  x = d^T
    A_matrix_for_d = np.eye(num_states) - current_gamma * P_pi_ss_prime.T
    b_vector_for_d = (1 - current_gamma) * initial_dist_mu_s
    
    try:
        d_P_pi_s_transposed = np.linalg.solve(A_matrix_for_d, b_vector_for_d)
        d_P_pi_s = d_P_pi_s_transposed # solve は列ベクトルとして解を返すので、これがd_P_pi(s)
    except np.linalg.LinAlgError:
        print("Warning: Linear system solve failed for occupancy measure. Using pseudo-inverse.")
        try:
            d_P_pi_s_transposed = np.linalg.pinv(A_matrix_for_d) @ b_vector_for_d
            d_P_pi_s = d_P_pi_s_transposed
        except np.linalg.LinAlgError:
            print("Error: Pseudo-inverse also failed for occupancy measure. Returning uniform distribution.")
            d_P_pi_s = np.ones(num_states) / num_states # フォールバック
    return d_P_pi_s # S次元のベクトル

def compute_gradient_J_c_U(
    P,            
    initial_dist_mu_s,      
    num_states,
    num_actions,
    pi,
    current_gamma,
    Q,
    H,
    index_U,
    jindex_c
):
    
    d_P_worst_pi_s = compute_occupancy_measure_d_P_pi(
        P[index_U], current_gamma, initial_dist_mu_s, num_states, num_actions, pi)
    
    gradient_J = np.zeros((S,A))  
    # 勾配の計算をします．最悪ケースの制約nおよびPに対して計算を行います．
    # d_P_worst_pi_sとHおよび，worstなPおよびcに対するQ行列を使って勾配を計算します．
    # ∇J = H * d_P_worst_pi_s * (Q[index_c][index_U])  # 勾配の計算
    for s_idx in range(num_states):
        for a_idx in range(num_actions):
            gradient_J[s_idx, a_idx] = H * d_P_worst_pi_s[s_idx] * Q[jindex_c][index_U][s_idx, a_idx]
    return gradient_J  


#　射影による方策更新(ここだけは北村さんのコード+GPTでnumpy用に生成しました．でもjaxにしたほうがいいと思う)
def projection_to_simplex_numpy(y_vector):
    """
    ベクトル y_vector を確率単体（要素が非負で合計が1）に射影します。
    JAXのチュートリアルなどで見られるアルゴリズムのNumPy版。

    Args:
        y_vector (np.ndarray): 射影したい1次元ベクトル。

    Returns:
        np.ndarray: 射影された1次元ベクトル。
    """
    n_features = y_vector.shape[0]
    u = np.sort(y_vector)[::-1]
    cssv = np.cumsum(u)
    indices = np.arange(n_features) + 1
    condition_for_rho = u + (1 - cssv) / indices > 0
    
    if not np.any(condition_for_rho):
        if np.all(y_vector >= 0) and np.isclose(np.sum(y_vector), 1.0):
            return y_vector
        rho = 0
    else:
        k_vals = np.arange(1, n_features + 1)
        sum_u_k = np.cumsum(u)
        condition_duchi_rho = u > (sum_u_k - 1) / k_vals
        if np.any(condition_duchi_rho):
            rho_duchi_0_indexed = np.where(condition_duchi_rho)[0][-1]
            # lambda_val (Duchiのtheta)
            lambda_val = (sum_u_k[rho_duchi_0_indexed] - 1) / (rho_duchi_0_indexed + 1)
            projected_y = np.maximum(y_vector - lambda_val, 0.)
            return projected_y
        else:
            y_non_negative = np.maximum(y_vector, 0)
            s = np.sum(y_non_negative)
            if s > 0:
                return y_non_negative / s
            else: # 全て0以下なら均等分布
                return np.ones(n_features) / n_features


def proj_to_Pi_numpy(policy_matrix_SA):
    """
    方策行列の各行（各状態の方策ベクトル）を確率単体に射影します。
    Args:
        policy_matrix_SA (np.ndarray): (S, A) の方策行列。

    Returns:
        np.ndarray: (S, A) の射影された方策行列。
    """
    if policy_matrix_SA.ndim != 2:
        raise ValueError("Input policy_matrix_SA must be a 2D array (S, A).")
    
    projected_policy = np.zeros_like(policy_matrix_SA)
    num_states = policy_matrix_SA.shape[0]
    
    for s_idx in range(num_states):
        projected_policy[s_idx, :] = projection_to_simplex_numpy(policy_matrix_SA[s_idx, :])
    return projected_policy

def update_and_project_policy(
    current_policy_table_SA,
    gradient_table_SA,       
    learning_rate_alpha
    ):
    """
    方策を勾配法で更新し、結果を確率単体に射影します。

    Args:
        current_policy_table_SA (np.ndarray): 現在の方策遷移行列 (S, A)。
        gradient_table_SA (np.ndarray): 計算された勾配行列 (S, A)。
        learning_rate_alpha (float): 学習率。

    Returns:
        np.ndarray: 更新され射影された新しい方策遷移行列 (S, A)。
    """
    if current_policy_table_SA.shape != gradient_table_SA.shape:
        raise ValueError("Shape mismatch between current_policy_table_SA and gradient_table_SA.")

    y_table_SA = current_policy_table_SA - learning_rate_alpha * gradient_table_SA
    
    # 2. 各状態の方策を確率単体に射影
    updated_projected_policy_table_SA = proj_to_Pi_numpy(y_table_SA)
    
    return updated_projected_policy_table_SA



def compute_inner_loop(b_0,pi):
    B[0]= b_0  # Bの最初の要素を更新
    for t in range(T):
        J_results, J_max_index,J_max_index_U = compute_J_c_U_b_and_its_max(U, C, Q_initial_list, B, pi)  # 各不確実性集合に対して目的関数を計算
        Q_array = compute_Q_pi_for_all_U(C, Q_initial_list,U,pi)  # 各不確実性集合に対するQ行列を計算
        gradient_J = compute_gradient_J_c_U(
            U, mu, S, A, pi, gamma, Q_array, H, J_max_index_U[J_max_index], J_max_index
        )  # 勾配を計算
        pi = update_and_project_policy(pi.reshape((S,A)), gradient_J, alpha)  # 方策を更新し射影
        pi = pi.flatten()  # 1次元ベクトルに戻す
        # Qの最大値の推移を記録
        if t % 10 == 0:  # 10エピソードごとに出力
            print(f"Episode {t}: J_max = {J_results[J_max_index]:.4f}, pi = {pi}")
            print(f"Q_max for U[{J_max_index_U[J_max_index]}]: {np.max(Q_array[J_max_index, J_max_index_U[J_max_index]]):.4f}")
            print(f"Gradient at episode {t}: {gradient_J}")
    return pi


def compute_delta(pi,B):
    """
    deltaを計算します。
    """
    J_results, J_max_index, J_max_index_U = compute_J_c_U_b_and_its_max(U, C, Q_initial_list, B, pi)
    return J_results[J_max_index] - B[J_max_index] 

def compute_i_j(i_curren,j_curren,delta,B):
    """
    iとjを計算します。
    iは、deltaが正のときはb0，0以下のときはi(そのまま)に設定されます。
    jは、deltaが正のときはj(そのまま)、0以下のときはb0に設定されます。
    """
    if delta > 0:
        i_new = B[0]  # b0
        j_new = j_curren  # jはそのまま
    else:
        i_new = i_curren  # iはそのまま
        j_new = B[0]  # b0
    return i_new, j_new

i_current = 0  
j_current = H

B[0]= (i_current + j_current) / 2  

def compute_outer_loop():
    global pi_new, i_current, j_current, B,pi
    for t in range(T):
        pi = compute_inner_loop(B[0],pi)
        delta = compute_delta(pi, B)
        i_current, j_current = compute_i_j(i_current, j_current, delta, B)
        B[0] = (i_current + j_current) / 2
        print(f"Outer loop iteration {t}: i = {i_current}, j = {j_current}, delta = {delta:.4f}, B[0] = {B[0]:.4f}")


    return pi_new

compute_outer_loop()  # 外側ループを実行

Episode 0: J_max = 0.7573, pi = [0.71013798 0.28986202 0.44089633 0.55910367 0.42569381 0.57430619]
Q_max for U[2]: 10.9563
Gradient at episode 0: [[84.22478515 82.7409113 ]
 [69.43664779 70.99322605]
 [59.61538352 58.74078816]]
Episode 10: J_max = 0.4935, pi = [0.63779981 0.36220019 0.51727738 0.48272262 0.38043021 0.61956979]
Q_max for U[2]: 10.7050
Gradient at episode 10: [[80.54958016 79.13135427]
 [65.93579233 67.44049378]
 [61.71713906 60.78779845]]
Episode 20: J_max = 0.3945, pi = [0.59406856 0.40593144 0.50311639 0.49688361 0.31922915 0.68077085]
Q_max for U[2]: 10.6131
Gradient at episode 20: [[80.25592757 78.80033591]
 [63.91362374 65.38590637]
 [62.17827349 61.22691193]]
Episode 30: J_max = 0.2607, pi = [0.54356292 0.45643708 0.50419557 0.49580443 0.25877421 0.74122579]
Q_max for U[2]: 10.4878
Gradient at episode 30: [[79.39559343 77.9182053 ]
 [61.5711269  63.00575033]
 [62.85553147 61.8753584 ]]
Episode 40: J_max = 0.1245, pi = [0.49755729 0.50244271 0.48472513 0.51527487 

KeyboardInterrupt: 

## 得られた知見
内側ルーチンでのベルマン方程式の計算は，収束するまで反復的に計算をしている．
* 価値反復法は1反復で1ステップの更新だけ
* 方策反復法は1反復で収束するまで反復更新をする

ホライゾンが1に近いと，収束に時間がかかる＋勾配がめっちゃでかくなる

### 内側ループ
* T回反復する
  * 現在の方策を用いて方策評価を行う
      * 制約ごとに，J_n_U - bnを求める
        * 遷移確率ごとに，J_c_Pを計算する
          * ベルマン方程式を，Qの更新差分が収束するまで繰り返し適用する
        * 最もJ_c_Pが大きなPとJn_Uを求める．
        * J_n_U - bnを求める．
        * 最もJ_n_U - bnが大きくなる制約nを求める
  * 射影勾配法を用いて方策を更新する
    * 方策評価で求めた制約nにおけるJ_n_Uを現在の方策での劣勾配を求める．
      * ∇J_n_U(pi) = H * d_pi * Q (勾配の計算が怪しい)
      * (劣)勾配は方策と同じ次元
    * 方策から勾配を定数倍したものを引く
    * 引いたものを入力とする最適化(最小化)問題を解く
      * 最適化問題を最小化するような方策を求める
  * 方策を更新し，現在の方策とする．
  * 方策は，J_n_U - bnが最も小さくなったtにおける方策とする

### 外側ループ
i =0,j = Hとして初期化
* K回反復する
  * b_0 = (i+j)/2
  * b0を入力として，内側ループで得られた方策を獲得する
  * 得られた方策を用いて，J_n_U -bnの最大値を求める
  * 最大値を用いてiとjを更新する．
    * 最大値が0超過なら iはb0，それ以外ならそのまま
    * 最大値が0超過なら jはそのまま，それ以外ならb0
